# To calculate daily returns for a list of symbols at day 0

In [1]:
import pandas as pd
import os
import numpy

def ends(df,x=1):
    dfends=df.head(x).append(df.tail(x))
    return dfends 

In [ ]:
# Read the cleaned data and sort by symbol and by data, to make sure the date is in right order for any analysis

data_df=pd.read_csv(os.path.join('C:/Users/jack.jin/Documents/Market', 'SP500_FIVE_YR_Cleaned_Sorted.csv'),sep=',')

In [3]:
ends(data_df,2)

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Row Count
0,2015-01-02,41.180000,41.310001,40.369999,40.560001,38.733665,1529200.0,A,1246
1,2015-01-05,40.320000,40.459999,39.700001,39.799999,38.007896,2041800.0,A,1246
609292,2019-12-11,123.199997,123.709999,122.809998,123.559998,123.559998,1803000.0,ZTS,1246
609293,2019-12-12,123.690002,125.000000,123.389999,123.599998,123.599998,1069232.0,ZTS,1246


In [4]:
#https://pandas.pydata.org/pandas-docs/stable/user_guide/computation.html#expanding-windows

# Define Func to calculate daily changes:
def stock_daily_change(df):
    
    # Get the unique symbol list
    #symbol_list = list(set(stock_df['Symbol'])) this line works too but it will change the order.
    symbol_list = pd.unique(df['Symbol']).tolist()
    
    # Create an empty dataframe to host to-be processed data
    combo_df=pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Symbol','Row Count','Adj Close Change'])
    
    for sym in symbol_list:
        
        indiv_sym_df=df.loc[df['Symbol'] == sym,]
        adj_cl_pct=pd.DataFrame.pct_change(indiv_sym_df[['Adj Close']]) 
        adj_cl_pct=adj_cl_pct.rename(columns={'Adj Close':'Adj Close Change'})
        
        indiv_sym_df=indiv_sym_df.join(adj_cl_pct)
        combo_df=combo_df.append(indiv_sym_df)
    
    return combo_df 

In [8]:
# Apply the func to calculate daily changes:

stock_with_pct=stock_daily_change(data_df)

In [9]:
ends(stock_with_pct,2)

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Row Count,Adj Close Change
0,2015-01-02,41.180000,41.310001,40.369999,40.560001,38.733665,1529200.0,A,1246,NaN
1,2015-01-05,40.320000,40.459999,39.700001,39.799999,38.007896,2041800.0,A,1246,-0.018737
609292,2019-12-11,123.199997,123.709999,122.809998,123.559998,123.559998,1803000.0,ZTS,1246,0.006353
609293,2019-12-12,123.690002,125.000000,123.389999,123.599998,123.599998,1069232.0,ZTS,1246,0.000324


In [10]:
# Drop the first value (NaN) 

stock_with_pct.dropna(how='any', inplace=True)

In [11]:
# NaN dropped

ends(stock_with_pct,2)

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Row Count,Adj Close Change
1,2015-01-05,40.320000,40.459999,39.700001,39.799999,38.007896,2041800.0,A,1246,-0.018737
2,2015-01-06,39.810001,40.020000,39.020000,39.180000,37.415798,2080600.0,A,1246,-0.015578
609292,2019-12-11,123.199997,123.709999,122.809998,123.559998,123.559998,1803000.0,ZTS,1246,0.006353
609293,2019-12-12,123.690002,125.000000,123.389999,123.599998,123.599998,1069232.0,ZTS,1246,0.000324


In [12]:
stock_with_pct.to_csv(os.path.join('C:/Users/jack.jin/Documents/Market', 'SP500_FIVE_YR_Cleaned_Sorted_t0_PCT.csv'),sep=',', index=False)